# 1. đang chạy ngon, lúc đóng gói lại lăn đùng giở chứng 

- run POSTMAN thì ngon curl --location 'http://localhost:3000/get_video_data?url=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F10qe6DkNX6up7-oG69HRG3B4j1A-WiI_o%2Fview'

- mà cho @script.js   xài click vào url video lại không được . Báo lỗi như trong ảnh

I can see the error messages from your screenshot. The main issues appear to be:
1. CORS/Content Security Policy issues with Google Drive frames
2. Error in `loadSavedScores` function where scores.forEach is not a function
3. Connection issues

Let's fix these issues in your code. Here are the necessary changes:

```javascript:deploy1.3.4_combine_update/frontend/script.js
// ... existing code ...

async function loadSavedScores(videoUrl) {
    try {
        const encodedUrl = encodeURIComponent(videoUrl);
        const response = await fetch(`http://localhost:3000/get_scores?url_video=${encodedUrl}`);
        const scores = await response.json();
        
        // Add check to ensure scores is an array
        if (Array.isArray(scores)) {
            scores.forEach(score => {
                displayResults(score.criteria, score.note, score.score);
            });
        } else {
            console.warn('Received scores is not an array:', scores);
        }
    } catch (error) {
        console.error('Error loading saved scores:', error);
        showNotification('warning', 'Error loading saved scores');
    }
}

// Update the video loading section
document.getElementById('loadVideo').addEventListener('click', async () => {
    const videoLink = document.getElementById('videoLink').value.trim();
    currentVideoUrl = videoLink;
    
    try {
        // Add error handling for invalid URL format
        const fileIdMatch = videoLink.match(/\/d\/(.+?)\//);
        if (!fileIdMatch) {
            throw new Error('Invalid Google Drive URL format');
        }
        const fileId = fileIdMatch[1];

        // Add headers to handle CORS
        const response = await fetch(`http://localhost:3000/get_video_data?url=${encodeURIComponent(videoLink)}`, {
            headers: {
                'Accept': 'application/json',
                'Content-Type': 'application/json'
            }
        });

        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        
        // Update video preview with sandbox attribute
        const previewLink = `https://drive.google.com/file/d/${fileId}/preview`;
        const videoFrame = document.getElementById('video');
        videoFrame.src = previewLink;
        videoFrame.setAttribute('sandbox', 'allow-same-origin allow-scripts');
        
        // Continue with rest of the function
        updateTranscript(data.transcript);
        const criteriaObj = JSON.parse(data.criteria);
        addCriteriaListeners(criteriaObj, fileId);
        await loadSavedScores(currentVideoUrl);
        
    } catch (error) {
        console.error('Error:', error);
        showNotification('warning', `Error loading video data: ${error.message}`);
    }
});
// ... existing code ...
```

The changes include:

1. Added better error handling in `loadSavedScores` to check if scores is actually an array before calling forEach
2. Updated the video loading event listener with:
   - Better URL validation
   - CORS headers
   - Proper error handling
   - Added sandbox attribute to the iframe to help with Content Security Policy issues
3. Added proper URL encoding for API calls

Also, make sure your backend (Node.js server) has CORS enabled. Add this to your backend server:

```javascript:backend/server.js
const cors = require('cors');
app.use(cors());
```

And ensure you have the proper response headers in your backend API endpoints:

```javascript:backend/server.js
res.header('Access-Control-Allow-Origin', '*');
res.header('Access-Control-Allow-Headers', 'Origin, X-Requested-With, Content-Type, Accept');
```

These changes should help resolve the connection and CORS issues you're experiencing.